In [2]:
# for loading/processing the images  
from tensorflow.keras.preprocessing.image import load_img 
from tensorflow.keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
import json
import matplotlib.backends.backend_pdf
import shutil

In [4]:
from glob import glob

In [3]:
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

2022-08-10 01:42:49.862385: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-08-10 01:42:49.862411: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-08-10 01:42:49.862725: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
data_path = '/mnt/largedrive0/katariap/feature_extraction/data/Dataset/Images_Tiled'
temp = glob(data_path+'/**/*.png',recursive = True)
print(len(temp))
print(temp[0])

418500
/mnt/largedrive0/katariap/feature_extraction/data/Dataset/Images_Tiled/Sample 483.vsi - 20x/Sample 483.vsi - 20x [x=52000,y=28000,w=1000,h=1000].png


In [7]:
def image_feature_extractor(path_tiled,model):

    patches = glob(path_tiled+'/**/*.png',recursive = True)
    print("Number of Patches in the dataset are : {}".format(len(patches)))
    feature_dictionary = {}

    i = 0
    for patch in patches:
        try:
            if(os.path.getsize(patch) > 0):
                image = load_img(patch,target_size=(224,224))
                image = np.array(image)
                reshaped_image = image.reshape(1,224,224,3)
                image_processed = preprocess_input(reshaped_image)
                feature_vector = model.predict(image_processed, use_multiprocessing = True)
                feature_dictionary[patch] = feature_vector[0].tolist()
        except:
            print(patch)
            continue
        if(i % 100 == 0):
            print("Features Extrated of {} Patches".format(i))
        i = i + 1
    print('Feature Extraction for dataset Done')
    return feature_dictionary

In [8]:
extracted_features = image_feature_extractor('/mnt/largedrive0/katariap/feature_extraction/data/Dataset/Images_Tiled',model)

Number of Patches in the dataset are : 418500
1/1 [==============================] - 0s 439ms/step
Features Extrated of 0 Patches
1/1 [==============================] - 0s 130ms/step
Features Extrated of 100 Patches
1/1 [==============================] - 0s 130ms/step
Features Extrated of 200 Patches
1/1 [==============================] - 0s 143ms/step
Features Extrated of 300 Patches
1/1 [==============================] - 0s 137ms/step
Features Extrated of 400 Patches
1/1 [==============================] - 0s 134ms/step
Features Extrated of 500 Patches
1/1 [==============================] - 0s 144ms/step
Features Extrated of 600 Patches
1/1 [==============================] - 0s 149ms/step
Features Extrated of 700 Patches
1/1 [==============================] - 0s 137ms/step
Features Extrated of 800 Patches
1/1 [==============================] - 0s 130ms/step
Features Extrated of 900 Patches
1/1 [==============================] - 0s 147ms/step
Features Extrated of 1000 Patches
1/1 [====

In [ ]:
with open('/mnt/largedrive0/katariap/feature_extraction/data/Dataset/VGG_features.json',"w") as file:
            file.write(json.dumps(extracted_features)) 

In [ ]:
number_of_clusters = 20

In [ ]:
def cluster_patches(feature_vector_file,number_of_clusters):

   
    feature_dictionary = {}
    print(feature_vector_file)
    with open(feature_vector_file,"r") as file:
        feature_dictionary = json.loads(file.read())
    feature_values = np.array(list(feature_dictionary.values()))
    file_names = np.array(list(feature_dictionary.keys()))

    pca = PCA(n_components=500,random_state=23)
    pca.fit(feature_values)
    values_transformed = pca.transform(feature_values)

    kmeans = KMeans(n_clusters=number_of_clusters)
    kmeans.fit(values_transformed)

    data_frame = pd.DataFrame(file_names,columns = ['filename'])
    data_frame['Cluster'] = kmeans.labels_


    name = '/mnt/largedrive0/katariap/feature_extraction/data/Dataset/vgg_clusters.csv'
    data_frame.to_csv(name)

    clusters = {}
    for file,cluster in zip(file_names,kmeans.labels_):
        if cluster not in clusters.keys():
            clusters[cluster] = []
            clusters[cluster].append(file)
        else:
            clusters[cluster].append(file)

    with open('/mnt/largedrive0/katariap/feature_extraction/data/Dataset/Dataset_clusters.pickle', 'wb') as file:
        pickle.dump(clusters, file)